In [2]:
import re
import time
import csv
from urllib.parse import urljoin

import requests
from bs4 import BeautifulSoup
from requests.adapters import HTTPAdapter, Retry

BASE = "https://www.aselsan.com"
LIST_URL = "https://www.aselsan.com/tr/haberler"

# Tek bir oturum kullan (daha hızlı ve nazik) + retry/backoff
session = requests.Session()
session.headers.update({
    "User-Agent": "Mozilla/5.0 (compatible; aselsan-scraper/1.0; +https://example.com)",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Accept-Language": "tr-TR,tr;q=0.9,en-US;q=0.8,en;q=0.7",
    "Referer": LIST_URL,
})
retries = Retry(
    total=5,
    connect=5,
    read=5,
    backoff_factor=0.6,
    status_forcelist=[429, 500, 502, 503, 504],
    allowed_methods=["HEAD", "GET", "OPTIONS"]
)
session.mount("https://", HTTPAdapter(max_retries=retries))
session.mount("http://", HTTPAdapter(max_retries=retries))


def _get_soup(url):
    r = session.get(url, timeout=30)
    r.raise_for_status()
    return BeautifulSoup(r.text, "html.parser")


def list_page_urls(page_no=1):
    """
    Haber liste sayfasındaki detay linklerini döndürür.
    Sayfa yoksa boş liste döner.
    Bazı sitelerde farklı sayfalama parametreleri olabildiği için
    birkaç varyasyonu deniyoruz.
    """
    candidate_urls = []
    if page_no == 1:
        candidate_urls.append(LIST_URL)
    else:
        candidate_urls.extend([
            f"{LIST_URL}?pageNo={page_no}",
            f"{LIST_URL}?page={page_no}",
            f"{LIST_URL}?p={page_no}",
        ])

    links = []
    for url in candidate_urls:
        try:
            soup = _get_soup(url)
        except requests.HTTPError:
            continue
        except Exception:
            continue

        # Liste içinde detay linkleri
        for a in soup.select('a[href*="/haberler/detay/"]'):
            href = a.get("href")
            if not href:
                continue
            full = urljoin(BASE, href)
            links.append(full)

        # Eğer bağlantı bulduysak diğer varyasyonları denemeye gerek yok
        if links:
            break

    return sorted(set(links))


def _text_or_none(tag):
    return tag.get_text(strip=True) if tag else None


def _pick_date_from_meta(soup):
    # Fallback: meta[property="article:published_time"] veya benzeri
    meta = soup.find("meta", attrs={"property": re.compile(r"(article:published_time|og:updated_time)")})
    if meta and meta.get("content"):
        return meta["content"]
    # Sayfada tarih formatı: 01.09.2025 / 1 Eylül 2025 gibi
    text = soup.get_text(" ", strip=True)
    m = re.search(r"\b(\d{1,2}\.\d{1,2}\.\d{4})\b", text)
    if m:
        return m.group(1)
    # Türkçe ay isimleri ile basit arama
    months = "Ocak|Şubat|Mart|Nisan|Mayıs|Haziran|Temmuz|Ağustos|Eylül|Ekim|Kasım|Aralık"
    m2 = re.search(rf"\b(\d{{1,2}}\s+(?:{months})\s+\d{{4}})\b", text)
    return m2.group(1) if m2 else None


def _pick_date_box(soup):
    """Takvim ikonlu tarih kutusunu bulur (varsa)."""
    for box in soup.select(".box.box-date, .news-date, .date, .meta, .info"):
        icon = box.find("i", class_=re.compile(r"fa-.*calendar"))
        if icon:
            # Kutudaki tarih benzeri ifadeyi izole etmeye çalış
            txt = box.get_text(" ", strip=True)
            # Önce dd.mm.yyyy
            m = re.search(r"\b\d{1,2}\.\d{1,2}\.\d{4}\b", txt)
            if m:
                return m.group(0)
            # Sonra ay ismi ile
            months = "Ocak|Şubat|Mart|Nisan|Mayıs|Haziran|Temmuz|Ağustos|Eylül|Ekim|Kasım|Aralık"
            m2 = re.search(rf"\b\d{{1,2}}\s+(?:{months})\s+\d{{4}}\b", txt)
            if m2:
                return m2.group(0)
            return txt  # hiçbiri tutmazsa kutunun tamamı
    return None


def _pick_readtime_box(soup):
    """Saat ikonlu okuma süresi kutusunu bulur (varsa)."""
    for box in soup.select(".box.box-date, .news-date, .date, .meta, .info"):
        icon = box.find("i", class_=re.compile(r"fa-.*clock"))
        if icon:
            txt = box.get_text(" ", strip=True)
            # 'dk', 'dakika' gibi ifadeleri yakala
            m = re.search(r"(\d+\s*(?:dk|dakika))", txt, flags=re.IGNORECASE)
            return m.group(1) if m else txt
    # Fallback: tüm sayfada bir arama
    text = soup.get_text(" ", strip=True)
    m = re.search(r"(\d+\s*(?:dk|dakika))", text, flags=re.IGNORECASE)
    return m.group(1) if m else None


def _pick_title(soup):
    title_tag = soup.select_one("h1.hero-title, h1.page-title, h1.title, h1")
    if title_tag:
        return _text_or_none(title_tag)
    # meta og:title
    meta = soup.find("meta", attrs={"property": "og:title"})
    if meta and meta.get("content"):
        return meta["content"].strip()
    # h2 fallback
    h2 = soup.select_one(".box.box-content h2, h2")
    return _text_or_none(h2)


def _pick_hero_image(soup):
    # 1) belirgin hero id/class
    hero_img = soup.select_one("#news-hero picture img, #news-hero img, .news-hero img, .hero img")
    for img in [hero_img] if hero_img else []:
        src = img.get("src") or img.get("data-src") or img.get("data-original")
        if src:
            return urljoin(BASE, src)

    # 2) og:image
    meta = soup.find("meta", attrs={"property": "og:image"})
    if meta and meta.get("content"):
        return urljoin(BASE, meta["content"])

    # 3) içerikte ilk anlamlı görsel
    any_img = soup.select_one(".box.box-content img, article img, .content img, img")
    if any_img:
        src = any_img.get("src") or any_img.get("data-src") or any_img.get("data-original")
        if src:
            return urljoin(BASE, src)
    return None


def parse_detail(url):
    """
    Haber detay sayfasını parse eder.
    Dönen dict: id, slug, url, title, date, read_time, hero_image, content_text, content_images
    """
    soup = _get_soup(url)

    # id ve slug (URL'den)
    m = re.search(r"/haberler/detay/(\d+)/([^/?#]+)", url)
    news_id = int(m.group(1)) if m else None
    slug = m.group(2) if m else None

    # başlık
    title = _pick_title(soup)

    # tarih & okuma süresi
    date_text = _pick_date_box(soup) or _pick_date_from_meta(soup)
    read_time = _pick_readtime_box(soup)

    # kapak görseli
    hero_src = _pick_hero_image(soup)

    # içerik gövdesi: birkaç olası kapsayıcı
    content_box = (
        soup.select_one(".box.box-content")
        or soup.select_one("article")
        or soup.select_one(".content")
        or soup.select_one(".news-detail")
    )

    paragraphs = []
    content_images = []
    if content_box:
        for p in content_box.select("p"):
            txt = p.get_text(" ", strip=True)
            if txt:
                paragraphs.append(txt)

        for img in content_box.select("img"):
            src = img.get("src") or img.get("data-src") or img.get("data-original")
            if src:
                content_images.append(urljoin(BASE, src))

    content_text = "\n\n".join(paragraphs) if paragraphs else None

    return {
        "id": news_id,
        "slug": slug,
        "url": url,
        "title": title,
        "date": date_text,
        "read_time": read_time,
        "hero_image": hero_src,
        "content_text": content_text,
        "content_images": content_images,
    }


def crawl_all(max_pages=50, delay=0.8):
    """
    Tüm haberleri sayfa sayfa gezip parse eder.
    max_pages: en fazla kaç liste sayfası denensin
    delay: her istek arasında bekleme (saniye)
    """
    seen_urls = set()
    records = []

    for p in range(1, max_pages + 1):
        try:
            links = list_page_urls(p)
        except requests.HTTPError as e:
            print(f"[WARN] Liste sayfası {p} HTTP hatası: {e}")
            break
        except Exception as e:
            print(f"[WARN] Liste sayfası {p} beklenmeyen hata: {e}")
            break

        if not links:
            # daha fazla sayfa görünmüyor
            break

        new_in_page = 0
        for u in links:
            if u in seen_urls:
                continue
            seen_urls.add(u)

            try:
                rec = parse_detail(u)
                records.append(rec)
                new_in_page += 1
            except requests.HTTPError as e:
                print(f"[WARN] Detay HTTP hatası: {u} -> {e}")
            except Exception as e:
                print(f"[WARN] Detay beklenmeyen hata: {u} -> {e}")

            time.sleep(delay)

        # Güvenlik: bir sayfada hiç yeni link yoksa erken çık
        if new_in_page == 0 and p > 1:
            break

    return records


def _sanitize_for_csv(s):
    if s is None:
        return ""
    # CSV’de düzenli görünmesi için olası \r\n ve dikey çizgi temizlikleri
    return s.replace("\r\n", "\n").replace("\r", "\n").strip()


if __name__ == "__main__":
    data = crawl_all(max_pages=60, delay=0.8)

    # id'ye göre benzersizleştir (son görüleni al)
    by_id = {}
    for item in data:
        if item and item.get("id") is not None:
            by_id[item["id"]] = item
    deduped = sorted(by_id.values(), key=lambda x: x["id"])

    # CSV yaz
    with open("aselsan_haberler.csv", "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(
            f,
            fieldnames=[
                "id",
                "slug",
                "url",
                "title",
                "date",
                "read_time",
                "hero_image",
                "content_text",
                "content_images",
            ],
        )
        writer.writeheader()
        for row in deduped:
            row_out = row.copy()
            # listeleri pipe ile birleştir
            imgs = row_out.get("content_images") or []
            row_out["content_images"] = "|".join(imgs)
            # metinleri temizle
            row_out["title"] = _sanitize_for_csv(row_out.get("title"))
            row_out["date"] = _sanitize_for_csv(row_out.get("date"))
            row_out["read_time"] = _sanitize_for_csv(row_out.get("read_time"))
            row_out["hero_image"] = _sanitize_for_csv(row_out.get("hero_image"))
            row_out["content_text"] = _sanitize_for_csv(row_out.get("content_text"))
            writer.writerow(row_out)

    print(f"{len(deduped)} haber kaydedildi -> aselsan_haberler.csv")

C:\Users\gundo\anaconda3\envs\py310\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


127 haber kaydedildi -> aselsan_haberler.csv
